# Ensemble NMS - Detectron2 [Inference]

### Hi kagglers, This is `Ensemble NMW - Detectron2 [Inference]` notebook.
* [Sartorius Segmentation - Detectron2 [training]](https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-detectron2-training) 
* [Sartorius Segmentation - Detectron2 [Inference]](https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-detectron2-inference) 
* [K-fold CrossValidation COCO Dataset Generator](https://www.kaggle.com/ammarnassanalhajali/k-fold-crossvalidation-coco-dataset-generator) 


### Please if this kernel is useful, <font color='red'>please upvote !!</font>

## Other notebooks in this competition 
- [Sartorius Segmentation - Keras U-Net[Training]](https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-keras-u-net-training)
- [Sartorius Segmentation - Keras U-Net[Inference]](https://www.kaggle.com/ammarnassanalhajali/sartorius-segmentation-keras-u-net-inference/edit)

# Intro
Ensembling multiple weaker performing models can help to get the results that you want.

## Install and import libraries

In [ ]:
!pip install ../input/detectron-05/whls/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/detectron2-0.5/detectron2 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/ensemble-boxes-104/ensemble_boxes-1.0.4/ -f ./ --no-index

In [ ]:
import os
import cv2
import json
import time
import numpy as np
import pandas as pd
import torch
import detectron2
from tqdm.auto import tqdm
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import inference_on_dataset
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.data import DatasetCatalog, build_detection_test_loader
import pycocotools.mask as mask_util
from skimage import morphology
from PIL import Image
import matplotlib.pyplot as plt
from fastcore.all import *
from ensemble_boxes import *
os.environ['CUDA_VISIBLE_DEVICES'] = '0' 
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    print('GPU is available')
else:
    DEVICE = torch.device('cpu')
    print('CPU is used')
print('detectron ver:', detectron2.__version__)

## My Models

In [ ]:
best_model=(
    {'file': 'model_best_32768_thr_35_55_75_CV32593.pth','config_name':'Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml', 'LB score': 0.338,'ths':[0.35, 0.55, 0.75]},
    {'file': 'model_cv3285_ths35_55_70.pth','config_name':'Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml', 'LB score': 0.337,'ths':[0.35, 0.55, 0.7]},
#     {'file': 'model_cv3278_ths5_65_75.pth','config_name':'Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml', 'LB score': 0.337,'ths':[0.5, 0.65, 0.75]},
#     {'file': 'model_best_32961_thr_35_5_8_CV32748.pth','config_name':'Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml', 'LB score': 0.337,'ths':[0.35, 0.5, 0.8]},
#     {'file': 'semi_r50_LB334_cv3234_thr26_38_61.pth','config_name':'Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml', 'LB score': 0.334,'ths':[0.26, 0.38, 0.61]}
            )

In [ ]:
#config_name = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
mdl_path = "../input/det2r152"
DATA_PATH = "../input/sartorius-cell-instance-segmentation"
MODELS = []
BEST_MODELS =[]
THSS = []
ID_TEST = 0
SUBM_PATH = f'{DATA_PATH}/test'
SINGLE_MODE = False
NMS = True
# MIN_PIXELS = [75, 150, 75]
MIN_PIXELS = [59, 136, 74]
IOU_TH = .4

for b_m in best_model:
    model_name=b_m["file"]
    model_ths=b_m["ths"]
    config_name=b_m["config_name"]
    BEST_MODELS.append(model_name)
    THSS.append(model_ths)
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(config_name))
    cfg.INPUT.MASK_FORMAT = 'bitmask'
    
    cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.2, 0.5, 1.0, 2.0, 5.0]]  # [[0.5, 1.0, 2.0]]
    cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[9], [18],[64], [120], [240]]
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 1024
    cfg.TEST.DETECTIONS_PER_IMAGE = 1000
    cfg.INPUT.MIN_SIZE_TEST = 1040
#     cfg.INPUT.MIN_SIZE_TEST = 1300
    cfg.INPUT.MAX_SIZE_TEST = 1408
#     cfg.INPUT.MAX_SIZE_TEST = 1760
    if 'semi_r50_LB334_cv3234_thr26_38_61' in model_name:
        cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.25, 0.5, 1.0, 2.0, 4.0]]
        cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[18], [34], [64], [128], [256]]
        cfg.INPUT.MIN_SIZE_TEST = 800
        cfg.INPUT.MAX_SIZE_TEST = 1333
        cfg.MODEL.WEIGHTS = '../input/det2r152/semi_r50_LB334_cv3234_thr26_38_61.pth'
    if 'model_cv3285_ths35_55_70' in model_name:
        cfg.MODEL.RESNETS.DEPTH = 101
        cfg.INPUT.MAX_SIZE_TEST= 1440
        cfg.INPUT.MIN_SIZE_TEST= 1040
#         cfg.INPUT.MAX_SIZE_TEST= 1760
#         cfg.INPUT.MIN_SIZE_TEST= 1300
        cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.2, 0.5, 1.0, 2.0, 5.0]]  # [[0.5, 1.0, 2.0]]
        cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[9], [17], [31], [64], [127]]
        cfg.MODEL.WEIGHTS = '../input/semidet2x152/semi_101/semi_puck_0439999/model_cv3285_ths35_55_70.pth'
    if 'model_best_32768_thr_35_55_75_CV32593' in model_name:
        cfg.MODEL.WEIGHTS = '../input/det2r152/model_best_32768_thr_35_55_75_CV32593.pth'
    if 'model_best_32961_thr_35_5_8_CV32748' in model_name:
        cfg.MODEL.WEIGHTS = '../input/det2r152/model_best_32961_thr_35_5_8_CV32748.pth'    
    if 'model_best_32845_thr_35_5_75_CV32724' in model_name:
        cfg.MODEL.WEIGHTS = '../input/det2r152/model_best_32845_thr_35_5_75_CV32724.pth'
    if 'model_cv3278_ths5_65_75' in model_name:
        cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.2, 0.5, 1.0, 2.0, 5.0]]  # [[0.5, 1.0, 2.0]]
        cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[9], [17], [31], [64], [127]]
        cfg.MODEL.WEIGHTS = '../input/semidet2x152/semi_new_anchor_1212/semi_new_1212_cmp_0/model_cv3278_ths5_65_75.pth'
    if 'model_best_33111_thr_45_5_8_CV32863' in model_name:
        cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.25, 0.5, 1.0, 2.0, 4.0]]  # [[0.5, 1.0, 2.0]]
        cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[18], [34], [64], [128], [256]]
        cfg.MODEL.WEIGHTS = '../input/det2r152/model_best_33111_thr_45_5_8_CV32863.pth'
    
#     cfg.MODEL.WEIGHTS = f'{mdl_path}/{model_name}'  
    MODELS.append(DefaultPredictor(cfg))
print(f'all loaded:\nthresholds: {THSS}\nmodels: {BEST_MODELS}')

In [ ]:
MODELS

## Utils

In [ ]:
def rle_decode(mask_rle, shape=(520, 704)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) 
                       for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def pred_masks(file_name, path, model, ths, min_pixels):
    img = cv2.imread(f'{path}/{file_name}')
    output = model(img)
    pred_classes = output['instances'].pred_classes.cpu().numpy().tolist()
    pred_class = max(set(pred_classes), key=pred_classes.count)
    take = output['instances'].scores >= ths[pred_class]
    pred_masks = output['instances'].pred_masks[take]
    pred_masks = pred_masks.cpu().numpy()
    result = []
    used = np.zeros(img.shape[:2], dtype=int) 
    for i, mask in enumerate(pred_masks):
        mask = mask * (1 - used)
        if mask.sum() >= min_pixels[pred_class]:
            used += mask
            result.append(rle_encode(mask))
    return result

def ensemble_preds(file_name, path, models, ths):
    img = cv2.imread(f'{path}/{file_name}')
    classes = []
    scores = []
    bboxes = []
    masks = []
    for i, model in enumerate(models):
        output = model(img)
        pred_classes = output['instances'].pred_classes.cpu().numpy().tolist()
        pred_class = max(set(pred_classes), key=pred_classes.count)
        take = output['instances'].scores >= ths[i][pred_class]
        classes.extend(output['instances'].pred_classes[take].cpu().numpy().tolist())
        scores.extend(output['instances'].scores[take].cpu().numpy().tolist())
        bboxes.extend(output['instances'].pred_boxes[take].tensor.cpu().numpy().tolist())
        masks.extend(output['instances'].pred_masks[take].cpu().numpy())
    assert len(classes) == len(masks) , 'ensemble lenght mismatch'
    #scores, classes, bboxes, masks = zip(*sorted(zip(scores, classes, bboxes, masks),reverse=True))
    return classes, scores, bboxes, masks

def nms_predictions(classes, scores, bboxes, masks, 
                    iou_th=.5, shape=(520, 704)):
    he, wd = shape[0], shape[1]
    boxes_list = [[[x[0] / wd, x[1] / he, x[2] / wd, x[3] / he] for x in bboxes]]
    scores_list = [[x for x in scores]]
    classes_list = [[x for x in classes]]
    nms_bboxes, nms_scores, nms_classes = non_maximum_weighted(
        boxes_list, 
        scores_list, 
        classes_list, 
        weights=None,
        iou_thr=0.5,skip_box_thr=0.0001  
    )
    nms_masks = []
    for s in nms_scores:
        nms_masks.append(masks[scores.index(s)])
    nms_scores, nms_classes, nms_masks = zip(*sorted(zip(nms_scores, nms_classes, nms_masks), reverse=True))
    return nms_classes, nms_scores, nms_masks

def ensemble_pred_masks(masks, classes, min_pixels, shape=(520, 704)):
    result = []
    #pred_class = max(set(classes), key=classes.count)
    pred_class = int(max(set(classes), key=classes.count).item())
    used = np.zeros(shape, dtype=int) 
    for i, mask in enumerate(masks):
        mask = mask * (1 - used)
        if mask.sum() >= min_pixels[pred_class]:
            used += mask
            result.append(rle_encode(mask))
    return result

def ensemble_pred_masks_val(masks, classes, min_pixels, shape=(520, 704)):
    result = []
    #pred_class = max(set(classes), key=classes.count)
    pred_class = int(max(set(classes), key=classes.count).item())
    used = np.zeros(shape, dtype=int) 
    for i, mask in enumerate(masks):
        mask = mask * (1 - used)
        if mask.sum() >= min_pixels[pred_class]:
            used += mask
            result.append(mask)
    return result

In [ ]:
# https://github.com/MouseLand/cellpose/blob/master/cellpose/utils.py
# https://scikit-image.org/docs/dev/api/skimage.morphology.html#skimage.morphology.remove_small_holes
def fill_small_hole(mask):
    hsz = np.count_nonzero(mask)*HOLE_SIZE_PERCENT #turn hole size into percentage
    padmsk = morphology.remove_small_holes(mask,hsz)
    return padmsk

In [ ]:
BOXES_THRESHOLDS_VOTE = [0.1, 0.1, 0.1]
IOU_THRESHOLDS_VOTE = [0.8,0.8,0.8]
# MASK_THRESHOLDS_VOTE = [0.4, 0.75, 0.75]
MASK_THRESHOLDS_VOTE = [0.1, 0.1, 0.1]
CELL_TYPES = {0: 'shsy5y', 1: 'astro', 2: 'cort'}
HOLE_SIZE_PERCENT = 0.1

In [ ]:
def compute_iou(box, boxes):
    # get area of a
    area_a = (box[3]-box[1])*(box[2]-box[0])
    # get area of b
    area_b = (boxes[:,3]-boxes[:,1])*(boxes[:,2]-boxes[:,0])

    # get left top x of IoU
    iou_x1 = np.maximum(boxes[:,0], box[0])
    # get left top y of IoU
    iou_y1 = np.maximum(boxes[:,1], box[1])
    # get right bottom of IoU
    iou_x2 = np.minimum(boxes[:,2], box[2])
    # get right bottom of IoU
    iou_y2 = np.minimum(boxes[:,3], box[3])

    # get width of IoU
    iou_w = iou_x2 - iou_x1
    iou_w = np.maximum(iou_w, 0)
    # get height of IoU
    iou_h = iou_y2 - iou_y1
    iou_h = np.maximum(iou_h, 0)
    
    # get area of IoU
    area_iou = iou_w * iou_h
    # get overlap ratio between IoU and all area
    iou = area_iou / (area_a + area_b - area_iou+1e-6)
    return iou
    
def process_preds_det2(pred):
    pred_class = torch.mode(pred['instances'].pred_classes)[0]
    take = pred['instances'].scores >= BOXES_THRESHOLDS_VOTE[pred_class]
    pred_masks = pred['instances'].pred_masks[take]
    mask_scores = pred['instances'].scores[take].cpu().numpy()
    boxes = pred['instances'].pred_boxes[take].tensor.cpu().numpy()
    pred_masks = pred_masks.cpu().numpy()
    pred_masks = mask_scores[:, np.newaxis, np.newaxis] * pred_masks
    nbox = len(boxes)
    out_masks = []
    for idx in range(nbox):
        box = boxes[idx]
        iou = compute_iou(box, boxes)
        take = iou>IOU_THRESHOLDS_VOTE[pred_class]
        mask = pred_masks[take].mean(axis=0)>=MASK_THRESHOLDS_VOTE[pred_class]
        mask = fill_small_hole(mask)
        out_masks.append(mask)
    return out_masks

def process_masks(boxes, pred_masks, mask_scores, classes):
    mask_scores = np.array(mask_scores)
    boxes = np.array(boxes)
    pred_masks = mask_scores[:, np.newaxis, np.newaxis] * pred_masks
    nbox = len(boxes)
    out_masks = []
    for idx in range(nbox):
        box = boxes[idx]
        pred_class = classes[idx]
        iou = compute_iou(box, boxes)
        take = iou>IOU_THRESHOLDS_VOTE[pred_class]
        mask = pred_masks[take].mean(axis=0)>=MASK_THRESHOLDS_VOTE[pred_class]
        mask = fill_small_hole(mask)
        out_masks.append(mask)
    return out_masks
def get_bounding_box(masks):
    bboxes = []
    for mask in masks:
        yindices = np.where(np.any(mask, axis=0))[0]
        xindices = np.where(np.any(mask, axis=1))[0]
        if yindices.shape[0]:
            y1, y2 = yindices[[0, -1]]
            x1, x2 = xindices[[0, -1]]
            y2 += 1
            x2 += 1
        else:
            y1, x1, y2, x2 = 0, 0, 0, 0
        bboxes.append((y1, x1, y2, x2))
    return bboxes

def nms_predictions_votes(classes, scores, bboxes, masks, 
                    iou_th=.5, shape=(520, 704)):
    # preprocess
    masks = process_masks(bboxes, masks, scores, classes)
    bboxes = get_bounding_box(masks)
    he, wd = shape[0], shape[1]
    boxes_list = [[[x[0] / wd, x[1] / he, x[2] / wd, x[3] / he] for x in bboxes]]
    scores_list = [[x for x in scores]]
    classes_list = [[x for x in classes]]
    nms_bboxes, nms_scores, nms_classes = non_maximum_weighted(
        boxes_list, 
        scores_list, 
        classes_list, 
        weights=None,
        iou_thr=0.3,skip_box_thr=0.0001  
    )
    nms_masks = []
    for s in nms_scores:
        nms_masks.append(masks[scores.index(s)])
    nms_scores, nms_classes, nms_masks = zip(*sorted(zip(nms_scores, nms_classes, nms_masks), reverse=True))
    return nms_classes, nms_scores, nms_masks

In [ ]:
nms_predictions = nms_predictions_votes

## NMS CV

In [ ]:
register_coco_instances('sartorius_val',{},'../input/jsonsartorius/3classes/annotations_val.json', 
                        '../input/sartorius-cell-instance-segmentation/')

val_ds = DatasetCatalog.get('sartorius_val')
VAL_PATH = f'{DATA_PATH}/train'

In [ ]:
def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)


# for val_name in tqdm(val_names):
def nms_cv_score():
    After_nms_CV = []
    for item in val_ds:
    #     im =  cv2.imread(item['file_name'].split('/')[4])
        classes, scores, bboxes, masks = ensemble_preds(
            file_name=item['file_name'].split('/')[4], 
            path=VAL_PATH, 
            models=MODELS, 
            ths=THSS
        )
        if NMS:
            classes, scores, masks = nms_predictions(
                classes, 
                scores, 
                bboxes, 
                masks, 
                iou_th=IOU_TH
            )

        encoded_masks = ensemble_pred_masks_val(
            masks, 
            classes, 
            min_pixels=MIN_PIXELS
        )

        enc_preds = [mask_util.encode(np.asarray(p, order='F', dtype='uint8')) for p in encoded_masks]
        enc_targs = list(map(lambda x:x['segmentation'], item['annotations']))
        ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
        prec = []
        for t in np.arange(0.5, 1.0, 0.05):
            tp, fp, fn = precision_at(t, ious)
            p = tp / (tp + fp + fn)
            prec.append(p)
        mean_pre = np.mean(prec)
        After_nms_CV.append(mean_pre)
    
    return np.mean(After_nms_CV)

In [ ]:
After_nms_CV_all = nms_cv_score()
After_nms_CV_all

In [ ]:
test_names = os.listdir(SUBM_PATH)
print('test images:', len(test_names))

In [ ]:
test_names

## Inference

In [ ]:
subm_ids, subm_masks = [], []
show_img =[]
i=0
for test_name in tqdm(test_names):
    if SINGLE_MODE:
        encoded_masks = pred_masks(
            test_name, 
            path=SUBM_PATH, 
            model=MODELS[0],
            ths=THSS[0],
            min_pixels=MIN_PIXELS
        )
    else:
        classes, scores, bboxes, masks = ensemble_preds(
            file_name=test_name, 
            path=SUBM_PATH, 
            models=MODELS, 
            ths=THSS
        )
        if NMS:
            classes, scores, masks = nms_predictions(
                classes, 
                scores, 
                bboxes, 
                masks, 
                iou_th=IOU_TH
            )
        encoded_masks = ensemble_pred_masks(
            masks, 
            classes, 
            min_pixels=MIN_PIXELS
        )
    if i<4:
        i+=1
        show_img.append(encoded_masks)
    for enc_mask in encoded_masks:
        subm_ids.append(test_name[:test_name.find('.')])
        subm_masks.append(enc_mask)

In [ ]:
dataDir=Path('../input/sartorius-cell-instance-segmentation')
test_image_names = (dataDir/'test').ls()
for j in range(3):
    _, axs = plt.subplots(1,2, figsize=(40,30))
    axs[1].imshow(cv2.imread(str(test_image_names[j])))
    axs[1].axis("off")
    mask = np.zeros((520, 704, 1))
    encoded_masks = show_img[j]
    for enc in encoded_masks:
        mask += rle_decode(enc, shape=(520, 704, 1))
    
    mask = mask.clip(0, 1)
    axs[0].imshow(mask)
    axs[0].axis("off")

In [ ]:
pd.DataFrame({
    'id': subm_ids, 
    'predicted': subm_masks
}).to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head()

# References
1. https://www.kaggle.com/vgarshin/detectron2-inference-with-ensemble-and-nms